#### hashMe() function — a helper to create a hash of a message or data

In [351]:
# Importing required libraries
import hashlib
import json
import sys

In [352]:
# Defining a helper function that wraps our hashing algorithm
def hashMe(msg=""):
    if type(msg) != str:
        msg = json.dumps(msg, sort_keys=True)
    
    return hashlib.sha256(str(msg).encode('utf-8')).hexdigest()

In [353]:
transaction = {"sender": "Alice", "recipient": "Bob", "amount": 10}
print(hashMe(transaction))

34bbbd0c0fa96527407730b8eda141d8815d540b464b02dfa0588b0c53382516


#### makeTransaction() funtion — generate exchanges between Alice and Bob

In [354]:
import random
random.seed(2) # Sets a fixed seed so results are predictable every time you run the code(good for debugging and reproducability)

def makeTransaction(maxValue = 1000): # This will create valid transactions in range of (1, maxValue)
    sign = int(random.getrandbits(1)) * 2 - 1 # This is will randomly choose between -1 or 1 (Picks randomly whether Alice pays Bob or Bob pays Alice)
    amount = random.randint(1, maxValue)
    aliceAmount = sign * amount
    bobAmount = -1 * aliceAmount # Ensures a balanced transaction, and zero-sum

    return {u'Alice': aliceAmount, u'Bob': bobAmount}

In [355]:
for _ in range(5):
    print(makeTransaction())

{'Alice': 884, 'Bob': -884}
{'Alice': 870, 'Bob': -870}
{'Alice': -94, 'Bob': 94}
{'Alice': -370, 'Bob': 370}
{'Alice': 174, 'Bob': -174}


In [356]:
# creating a large set of transactions, then chunking them into blocks
transactionBuffer = [makeTransaction() for _ in range(100)] # 100 transactions
transactionBuffer

[{'Alice': 829, 'Bob': -829},
 {'Alice': 875, 'Bob': -875},
 {'Alice': -258, 'Bob': 258},
 {'Alice': 218, 'Bob': -218},
 {'Alice': 37, 'Bob': -37},
 {'Alice': 698, 'Bob': -698},
 {'Alice': -442, 'Bob': 442},
 {'Alice': 403, 'Bob': -403},
 {'Alice': 741, 'Bob': -741},
 {'Alice': 522, 'Bob': -522},
 {'Alice': 381, 'Bob': -381},
 {'Alice': 959, 'Bob': -959},
 {'Alice': -515, 'Bob': 515},
 {'Alice': -923, 'Bob': 923},
 {'Alice': -892, 'Bob': 892},
 {'Alice': -373, 'Bob': 373},
 {'Alice': -955, 'Bob': 955},
 {'Alice': -930, 'Bob': 930},
 {'Alice': -434, 'Bob': 434},
 {'Alice': 906, 'Bob': -906},
 {'Alice': 169, 'Bob': -169},
 {'Alice': 182, 'Bob': -182},
 {'Alice': -237, 'Bob': 237},
 {'Alice': -181, 'Bob': 181},
 {'Alice': -178, 'Bob': 178},
 {'Alice': -523, 'Bob': 523},
 {'Alice': 369, 'Bob': -369},
 {'Alice': 527, 'Bob': -527},
 {'Alice': 574, 'Bob': -574},
 {'Alice': -916, 'Bob': 916},
 {'Alice': -816, 'Bob': 816},
 {'Alice': -753, 'Bob': 753},
 {'Alice': 929, 'Bob': -929},
 {'Alice': 7

#### updateState(transaction, state) funtion — Applying a transaction to account balance
##### This does not validate the transaction, only updates the states. (Only called if the transaction is valid)

In [357]:
def updateState(transaction, state):
    '''
    Inputs: 
        transaction: a dictionary with keys Alice and Bob, and values the amount of money they are sending to each other
        state: a dictionary with keys Alice and Bob, and values their current balance
    Outputs:
        state: a dictionary with keys Alice and Bob, and values their new balance after the transaction
        ** Additional users will be added to state if they are not already present
    '''

    state = state.copy() # Copy the state so we don't modify the original

    for key in transaction.keys():
        if key in state.keys():
            state[key] += transaction[key]
        else:
            state[key] = transaction[key]
    
    return state

In [358]:
state = {'Alice': 1000, 'Bob': 1000} # Initial state of the system
print(f"Initial State:{state}")
transaction = makeTransaction()
print(f"Transaction:{transaction}")

newState = updateState(transaction, state)
print(f"Updated State:{newState}")

Initial State:{'Alice': 1000, 'Bob': 1000}
Transaction:{'Alice': 22, 'Bob': -22}
Updated State:{'Alice': 1022, 'Bob': 978}


#### isValidTransaction(transaction, state) funtion — Validating the transaction

In [359]:
def isValidTransaction(transaction, state):
    '''
    Inputs: 
        transaction: a dictionary with keys Alice and Bob or anyone else, and values the amount of money they are sending to each other
        state: a dictionary with keys Alice and Bob or anyone else, and values their current balance
    Outputs:
        True if the transaction is valid, False otherwise
    '''
    if sum(transaction.values()) != 0:
        return False # Check if the transaction is zero-sum (No new tokens are created or destroyed)
    
    for key in transaction.keys():
        if key in state.keys():
            accountBalance = state[key]
        else:
            accountBalance = 0
        
        if accountBalance + transaction[key] < 0:
            return False
        # Check if the transaction is valid by ensuring that the sender has enough money to send
        # If the sender is not in the state, they are assumed to have 0 balance, so we check if they are sending more than 0

    return True # If all checks pass, the transaction is valid

In [360]:
state = {'Alice': 1000, 'Bob': 1000} # Initial state of the system
print(f"Initial State:{state}")

transaction1 = {'Alice': -800, 'Bob': 800} # Alice sends 800 to Bob
transaction2 = {'Alice': 1500, 'Bob': -1500} # Bob sends 1500 to Alice (invalid transaction since Bob doesn't have enough money)
transaction3 = {'Alice': -400, 'Bob': 600} # Alice sends 400 to Bob but Bob receives 600 (invalid transaction since we can't create or destroy tokens)
transaction4 = {'Alice': -500, 'Bob': 250, 'Lisa': 250} # Creating new users is valid as long as the rules are preserved

print(f"Transaction: {transaction1}, Valid: {isValidTransaction(transaction1,state)}")
print(f"Transaction: {transaction2}, Valid: {isValidTransaction(transaction2,state)}")
print(f"Transaction: {transaction3}, Valid: {isValidTransaction(transaction3,state)}")
print(f"Transaction: {transaction4}, Valid: {isValidTransaction(transaction4,state)}")

Initial State:{'Alice': 1000, 'Bob': 1000}
Transaction: {'Alice': -800, 'Bob': 800}, Valid: True
Transaction: {'Alice': 1500, 'Bob': -1500}, Valid: False
Transaction: {'Alice': -400, 'Bob': 600}, Valid: False
Transaction: {'Alice': -500, 'Bob': 250, 'Lisa': 250}, Valid: True


#### Building the Blockchain: From Transactions to Blocks

In [361]:
state = {'Alice': 2000, 'Bob': 2000} # Initial state of the system (This is like setting up the original balances in a new blockchain network)

genesisBlockTransactions = [state] # This is the first block in the blockchain, so we set the transactions to the initial state

genesisBlockContents = {
    'blockNumber': 0,
    'parentHash': None,
    'transactionCount': 1,
    'transactions': genesisBlockTransactions
}

genesisHash = hashMe(genesisBlockContents) # Generating SHA-256 hash of block's content. This makes is immutable. If anything changes, the hash changes

genesisBlock = {'hash': genesisHash, 'contents': genesisBlockContents} # Creating the actual block using a dictionary

genesisBlockStr = json.dumps(genesisBlock, sort_keys=True) # Serializing the block for later use (storing, printing or transmitting)

# This becomes the first element from which everything else will be linked.

In [362]:
# creating the blockchain
chain = [genesisBlock]

#### makeBlock(transaction, chain) function

In [363]:
def makeBlock(transactions, chain):
    '''
        
    '''
    parentBlock = chain[-1]
    parentHash = parentBlock['hash']
    blockNumber = parentBlock['contents']['blockNumber'] + 1
    transactionCount = len(transactions)

    blockContents = {
        'blockNumber': blockNumber,
        'parentHash': parentHash,
        'transactionCount': transactionCount,
        'transactions': transactions
    }

    blockHash = hashMe(blockContents)

    block = {'hash': blockHash, 'contents': blockContents}

    return block

#### Processing transactionBuffer into Blocks

In [ ]:
blockSizeLimit = 10 # This is the maximum number of transactions that can be in a block
# This is a simple way to limit the size of the block. In a real blockchain, this would be more complex and would 
# involve checking the size of the block in bytes and ensuring that it doesn't exceed a certain limit. This is just
# a simple example to illustrate the concept.

while len(transactionBuffer) > 0:
    bufferStartSize = len(transactionBuffer) # This is the size of the buffer before we start processing it

    # Gather a set of valid transacrions for the block
    transactionList = [] # This is the list of transactions that will be in the block

    while (len(transactionList) < blockSizeLimit) and (len(transactionBuffer) > 0): # While we haven't reached the block size limit and there are still transactions in the buffer
        newTransaction = transactionBuffer.pop() # Pop a transaction from the buffer

        transactionValidity = isValidTransaction(newTransaction, state) # Check if the transaction is valid
        if transactionValidity:
            transactionList.append(newTransaction) # If the transaction is valid, add it to the list of transactions for the block
            state = updateState(newTransaction, state)
        else:
            print(f"Transaction {newTransaction} is invalid and will be discarded")
            # If the transaction is invalid, discard it and move on to the next one
            continue

    # If we have a block of transactions, create a new block and add it to the chain
    newBlock = makeBlock(transactionList, chain) # Create a new block with the transactions we gathered
    chain.append(newBlock) # Add the new block to the chain
    print(f"Block {newBlock['contents']['blockNumber']} created with {len(transactionList)} transactions")

Transaction {'Alice': -689, 'Bob': 689} is invalid and will be discarded
Transaction {'Alice': -177, 'Bob': 177} is invalid and will be discarded
Transaction {'Alice': -372, 'Bob': 372} is invalid and will be discarded
Block 1 created with 10 transactions
Block 2 created with 10 transactions
Transaction {'Alice': 376, 'Bob': -376} is invalid and will be discarded
Transaction {'Alice': 520, 'Bob': -520} is invalid and will be discarded
Transaction {'Alice': 576, 'Bob': -576} is invalid and will be discarded
Transaction {'Alice': 852, 'Bob': -852} is invalid and will be discarded
Transaction {'Alice': 819, 'Bob': -819} is invalid and will be discarded
Block 3 created with 10 transactions
Block 4 created with 10 transactions
Transaction {'Alice': 816, 'Bob': -816} is invalid and will be discarded
Transaction {'Alice': 511, 'Bob': -511} is invalid and will be discarded
Block 5 created with 10 transactions
Transaction {'Alice': 410, 'Bob': -410} is invalid and will be discarded
Block 6 crea

In [365]:
chain[0] # This is the genesis block, which is the first block in the chain. It has no parent and contains the initial state of the system

{'hash': '0cfb911091d3f6d330c2dff1bab497a582f6b99e91d1cca475c1639728289c17',
 'contents': {'blockNumber': 0,
  'parentHash': None,
  'transactionCount': 1,
  'transactions': [{'Alice': 2000, 'Bob': 2000}]}}

In [366]:
chain[3]

{'hash': 'e4e0f5be863cfc886026ff06111b802006a471213585b6c3a9453a70a392ab87',
 'contents': {'blockNumber': 3,
  'parentHash': '3c764cbadc3a00980590b7285754fb04478554215bc6b9ace4e3302971eef3bb',
  'transactionCount': 10,
  'transactions': [{'Alice': 9, 'Bob': -9},
   {'Alice': 350, 'Bob': -350},
   {'Alice': -804, 'Bob': 804},
   {'Alice': 904, 'Bob': -904},
   {'Alice': 701, 'Bob': -701},
   {'Alice': -501, 'Bob': 501},
   {'Alice': -749, 'Bob': 749},
   {'Alice': 417, 'Bob': -417},
   {'Alice': 631, 'Bob': -631},
   {'Alice': -311, 'Bob': 311}]}}

In [367]:
state # This is the final state of the system after all transactions have been processed. It shows the final balances of all users in the system

{'Alice': 3401, 'Bob': 599}

#### checkBlockHash(block) - This function ensures block integrity

In [368]:
def checkBlockHash(block):
    '''
    Inputs:
        block: a block in the blockchain
    Outputs:
        None if the hash is valid, raises an exception if the hash is invalid

    This function checks if the hash of the block is valid by comparing it to the hash of the block's contents.
    This protect the blockchain from tampering.
    If anyone changes the content of a block(even one letter),the hash will change and the block will become invalid.
    This achieves immutability in the blockchain
    '''

    expectedHash = hashMe(block['contents']) # Hash the block contents to get the expected hash
    if expectedHash != block['hash']:
        raise Exception(f"Hash does not match for block {block['contents']['blockNumber']}. Expected {expectedHash}, got {block['hash']}")
    return

#### checkBlockValidity(block, parent, state) - This function ensures block is valid in context(considers parent block and system state)

In [369]:
def chechkBlockValidity(block, parent, state):
    '''
    This function is used to validate the blocks in the blockchain and ensure that they are valid before adding them to the chain.
    Inputs:
        block: a block in the blockchain
        parent: the parent block of the current block
        state: the current state of the system
    Outputs:
        state: the updated state of the system after processing the block
    
    This function checks if the block is valid by checking the following:
        1. The block number is the next block after the parent block
        2. The parent hash is correct (the hash of the parent block)
        3. The transactions in the block are valid (using the isValidTransaction function)
        4. The hash of the block is valid (using the checkBlockHash function)
    If any of these checks fail, an exception is raised.
    If all checks pass, the state is updated with the transactions in the block and the updated state is returned.
    '''

    parentNumber = parent['contents']['blockNumber'] # Get the block number of the parent block
    parentHash = parent['hash'] # Get the hash of the parent block
    blockNumber = block['contents']['blockNumber'] # Get the block number of the current block

    # Check transaction vallidity, throw an error if any transaction is invalid
    for transaction in block['contents']['transactions']:
        if isValidTransaction(transaction, state):
            state = updateState(transaction, state)
        else:
            raise Exception(f"Transaction {transaction} in block {blockNumber} is invalid")
        
    checkBlockHash(block) # Check if the hash of the block is valid

    if blockNumber != parentNumber + 1:
        raise Exception(f"Block {blockNumber} is not the next block after {parentNumber}.") # Check if the block number is correct
    
    if block['contents']['parentHash'] != parentHash:
        raise Exception(f"Block {blockNumber} does not have the correct parent hash. Expected {parentHash}, got {block['contents']['parentHash']}") # Check if the parent hash is correct
    
    return state
        

#### checkChain(chain) - Check the validity of the chain

In [370]:
def checkChain(chain):
    '''
        This function checks the entire blockchain for validity and consistency.
        Inputs:
            chain: a list of blocks in the blockchain
        Outputs:
            state: the final state of the system after processing all blocks in the chain
        
        This function checks the following:
            1. The chain is a list of blocks
            2. The genesis block is valid (using the checkBlockHash function)
            3. Each block in the chain is valid (using the chechkBlockValidity function)
        If any of these checks fail, return False.
        If all checks pass, return the final state of the system after processing all blocks in the chain.
    '''

    # Check if the chain is a list of blocks
    if type(chain) == str:
        try:
            chain = json.loads(chain) # If the chain is a string, convert it to a list of blocks
            assert type(chain) == list # Check if the chain is a list
        except:
            return False
    elif type(chain) != list:
        return False
    
    state = {} # Initialize the state of the system

    for transaction in chain[0]['contents']['transactions']:
        state = updateState(transaction, state)
    # Update the state with the transactions in the genesis block(Genesis block is assumed to be valid)

    checkBlockHash(chain[0]) # Check if the hash of the genesis block is valid

    parent = chain[0] # Set the parent block to the genesis block

    for block in chain[1:]: # Iterate through the rest of the blocks in the chain
        state = chechkBlockValidity(block, parent, state)
        # Check if the block is valid and update the state with the transactions in the block
        parent = block
        # Set the parent block to the current block for the next iteration

    return state # Return the final state of the system after processing all blocks in the chain

In [371]:
checkChain(chain) # Check the entire blockchain for validity and consistency
# This will return the final state of the system after processing all blocks in the chain

{'Alice': 3401, 'Bob': 599}

In [372]:
# Even if we are loading the chain from a text file, e.g. from backup or loading it for the first time, we can check the integrity of the chain and create the current state
chainAsText = json.dumps(chain,sort_keys=True)
checkChain(chainAsText)

{'Alice': 3401, 'Bob': 599}

In [373]:
# Manually tampering with a transaction in the chain to see if the checkChain function catches it
chain[1]['contents']['transactions'][0]['Alice'] = -10000 # This is an invalid transaction since Alice doesn't have enough money to send 10000 to Bob

# This will cause the checkChain function to raise an exception when we try to check the chain again
try:
    checkChain(chain)
except Exception as e:
    print(f"Exception: {e}")

Exception: Transaction {'Alice': -10000, 'Bob': 747} in block 1 is invalid


In [374]:
# breaking the parentHash reference in block 3
chain[1]['contents']['transactions'][0]['Alice'] = -341 # Reset the transaction to a valid one so we can check the chain again
chain[3]['contents']['parentHash'] = '1234567890' # This is an invalid parent hash since it doesn't match the hash of the parent block

# This will cause the checkChain function to raise an exception when we try to check the chain again
try:
    checkChain(chain)
except Exception as e:
    print(f"Exception: {e}")

Exception: Transaction {'Alice': -341, 'Bob': 747} in block 1 is invalid


In [375]:
chain[4]

{'hash': '6854cf855b4782fc1a722705a746e9f7ffab843195f4e1890656f7d0a5013a6c',
 'contents': {'blockNumber': 4,
  'parentHash': 'e4e0f5be863cfc886026ff06111b802006a471213585b6c3a9453a70a392ab87',
  'transactionCount': 10,
  'transactions': [{'Alice': 492, 'Bob': -492},
   {'Alice': -792, 'Bob': 792},
   {'Alice': 632, 'Bob': -632},
   {'Alice': -898, 'Bob': 898},
   {'Alice': 856, 'Bob': -856},
   {'Alice': -835, 'Bob': 835},
   {'Alice': -964, 'Bob': 964},
   {'Alice': -675, 'Bob': 675},
   {'Alice': 468, 'Bob': -468},
   {'Alice': 571, 'Bob': -571}]}}

In [376]:
# Breaking the block number continuity in block 4
chain[3]['contents']['parentHash'] = chain[2]['hash'] # Reset the parent hash to the correct value so we can check the chain again
chain[4]['contents']['blockNumber'] = chain[3]['contents']['blockNumber'] + 2 # 

# This will cause the checkChain function to raise an exception when we try to check the chain again
try:
    checkChain(chain)
except Exception as e:
    print(f"Exception: {e}")

Exception: Transaction {'Alice': -341, 'Bob': 747} in block 1 is invalid


#### Putting it Together: The final blockchain Architecture

##### On Node A (The miner node)

In [378]:
import copy

nodeBchain = copy.copy(chain) # This is a shallow copy of the chain, so it will not affect the original chain
nodeBtransactions = [makeTransaction() for _ in range(10)] # 10 transactions
newBlock = makeBlock(nodeBtransactions, nodeBchain) # Create a new block with the transactions we gathered

In [379]:
newBlock

{'hash': '7dbc1ffc8ecc83079ef1dc94db2222caafd5a9eb7eecbdf1ea88e7ae8f8524d7',
 'contents': {'blockNumber': 10,
  'parentHash': '8519c1fd641aff9787b41d3efeb498d2a3822f5b8a2d2a7e5ee20c55e09b875b',
  'transactionCount': 10,
  'transactions': [{'Alice': -262, 'Bob': 262},
   {'Alice': -833, 'Bob': 833},
   {'Alice': 161, 'Bob': -161},
   {'Alice': 189, 'Bob': -189},
   {'Alice': 709, 'Bob': -709},
   {'Alice': -395, 'Bob': 395},
   {'Alice': 45, 'Bob': -45},
   {'Alice': 254, 'Bob': -254},
   {'Alice': -995, 'Bob': 995},
   {'Alice': -5, 'Bob': 5}]}}

In [380]:
chechkBlockValidity(newBlock, nodeBchain[-1], state) # Check if the block is valid and update the state with the transactions in the block

{'Alice': 2269, 'Bob': 1731}

##### Assume the newBlock is received by our Node.

In [381]:
print(f"Blockchain on Node A is currently {len(chain)} blocks long")

try:
    print("New block received. Checking validity...")
    state = chechkBlockValidity(newBlock, chain[-1], state) # Update the state, but this will throw an error if the block is invalid
    chain.append(newBlock)

except:
    print("Invalid block. Ignoring and waiting for next block...")

print(f"Blockchain on Node A is now {len(chain)} blocks long")

Blockchain on Node A is currently 10 blocks long
New block received. Checking validity...
Blockchain on Node A is now 11 blocks long
